In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# 크롤링할 기간 설정
start_date = datetime(2023, 2, 1)
end_date = datetime(2023, 2, 2)

# 크롤링할 URL 설정
url = "https://news.daum.net/breakingnews/digital"

# 크롤링할 기사 링크를 저장할 리스트 초기화
links = set()

# 주어진 기간 동안 각 페이지의 기사 링크 추출
while start_date <= end_date:
    # 현재 날짜에 해당하는 페이지 URL 생성
    date_str = start_date.strftime("%Y%m%d")
    page_url = f"{url}?regDate={date_str}"
    num_page = 1
    last_page = False

    # 각 페이지의 기사 링크 추출
    while True:
        page_url_num = f"{page_url}&page={num_page}"
        res = requests.get(page_url_num)

        # HTTP 요청이 성공했는지 확인
        if res.status_code != 200:
            print(f"Failed to retrieve page: {page_url_num}")
            continue

        # BeautifulSoup으로 HTML 파싱
        soup = BeautifulSoup(res.content, "html.parser")

        # 각 기사의 링크를 추출하여 리스트에 추가
        article_links = soup.select(".list_news2 a.link_txt")

        if not article_links:
            break

        for link in article_links:
            article_url = link["href"]
            article_res = requests.get(article_url)

            # HTTP 요청이 성공했는지 확인
            if article_res.status_code != 200:
                print(f"Failed to retrieve article: {article_url}")
                continue

            article_soup = BeautifulSoup(article_res.content, "html.parser")
            date = article_soup.select_one(".info_view .txt_info")

            if date is None:
                date = start_date.strftime("%Y.%m.%d.")
            else:
                date = date.get_text().strip()

            if (article_url, date) not in links:
                links.add((article_url, date))

        # 마지막 페이지 여부를 검사하여 last_page 변수에 할당
        last_page = (len(article_links) < 10)
        if last_page:
            break

        num_page += 1

    # 다음 페이지로 넘어가기 위해 날짜 1일 증가
    start_date += timedelta(days=1)


# 추출된 링크를 이용하여 기사 내용 크롤링
for link, date in links:
    # 기사 URL 출력 (테스트용)
    print(f"\n☞링크: {link}")

    # 기사 HTML 코드 가져오기
    res = requests.get(link)

    # HTTP 요청이 성공했는지 확인
    if res.status_code != 200:
        print(f"Failed to retrieve article: {link}")
        continue

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(res.content, "html.parser")

    # 기사 제목 추출
    title = soup.select_one(".tit_view").get_text()

    # 기사 본문 추출
    contents = soup.select(".article_view p")
    content = "\n".join([c.get_text().strip() for c in contents])

    # 추출한 제목과 내용 출력
    print(f"☞기사제목: {title}")
    print(f"☞작성일자: {date_str}")
    print(f"☞기사내용: \n{content}")

KeyboardInterrupt: 